In [1]:
import pandas as pd;
import os;
import re;
import pickle;

os.chdir("/Users/hannahkaufman/Downloads/Data/output");
os.getcwd()

'/Users/hannahkaufman/Downloads/Data/output'

In [25]:
files = os.listdir('./');
for f in files:
    if (f.endswith('.somatic.maf.short.gz')):
        print(f, '...')
        somatic_short = pd.read_csv(f, sep='\t', compression='gzip');
        prot_pos_parsed = somatic_short['prot.pos'].str.extract('(?P<prot_pos>[\\-\d]+)/(?P<prot_len>\d+)')
        aa_parsed = somatic_short['aa'].str.extract('(\w*)/*([\w\*]*)')
        aa_parsed.columns = ['ref_aa', 'alt_aa']
        patient_parsed = somatic_short['sample_id'].str.extract('(?P<patient>TCGA-[\w\d]+-[\w\d]+)-')
        somatic_short = somatic_short.join(prot_pos_parsed)
        somatic_short = somatic_short.join(aa_parsed)
        somatic_short = somatic_short.join(patient_parsed)
        
        f_clinical = f.replace('somatic.maf', 'clinical')
        f_clinical = f_clinical.replace('.gz', '.txt')
        clinical_short = pd.read_csv(f_clinical, sep='\t') 
        clinical_somatic = clinical_short.join(somatic_short.set_index('patient'), on='patient', how='inner')
        print('  merged (row, column):', clinical_somatic.shape);       

        f_output = f.replace('somatic.maf.short.gz', 'clinical_somatic.df.pkl')
        output_file = open(f_output, 'wb') # binary write
        pickle.dump([clinical_somatic], output_file)
        output_file.close()

LAML.somatic.maf.short.gz ...
  merged (row, column): (6716, 26)
SKCM.somatic.maf.short.gz ...
  merged (row, column): (336541, 26)
ACC.somatic.maf.short.gz ...
  merged (row, column): (5913, 26)
BRCA.somatic.maf.short.gz ...
  merged (row, column): (96963, 26)


In [26]:
clinical_short


,patient,year_of_birth,gender,race,definition,tumor_stage,age_at_diagnosis,vital_status,months_to_death,months_to_last_follow_up
0,TCGA-E2-A15D,1962.0,female,white,Primary solid Tumor,stage iia,47.2,alive,NaN,17.5
1,TCGA-C8-A131,1928.0,female,asian,Primary solid Tumor,stage iiia,82.1,alive,NaN,13.7
2,TCGA-BH-A18L,1953.0,female,white,Primary solid Tumor,stage iiia,50.7,dead,27.0,NaN
3,TCGA-B6-A0X4,1934.0,female,white,Primary solid Tumor,stage iib,62.2,dead,28.7,NaN
4,TCGA-BH-A0BF,1953.0,female,white,Primary solid Tumor,stage iib,56.7,dead,44.1,26.1
...,...,...,...,...,...,...,...,...,...,...
1104,TCGA-BH-A28Q,1964.0,female,white,Primary solid Tumor,stage iib,46.3,alive,NaN,37.3
1105,TCGA-A7-A3IZ,1949.0,female,white,Primary solid Tumor,stage iia,63.0,alive,NaN,10.7
1106,TCGA-A2-A1G0,1961.0,female,white,Primary solid Tumor,stage iib,49.3,alive,NaN,20.5
1107,TCGA-A2-A3XS,1941.0,female,black or african american,Primary solid Tumor,stage iiia,63.0,dead,34.4,NaN
